###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [ ]:
import sqlite3
from pathlib import Path
from google.colab import drive

#mount drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .

#create and save store.db
gdrive='drive/MyDrive/CS 133/Lec14/'
Path(gdrive+'store.db').touch()

conn = sqlite3.connect(gdrive+'store.db') # creates a connection to the database
c = conn.cursor() # the cursor is what we'll actually use

#create the tables
create_product_table = '''CREATE TABLE IF NOT EXISTS product (
             product_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             price REAL
             )'''

create_customer_table = '''CREATE TABLE IF NOT EXISTS customer (
             customer_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             email TEXT NOT NULL
             )'''

create_order_item_table = '''CREATE TABLE IF NOT EXISTS order_item (
             order_id INTEGER,
             product_id INTEGER,
             quantity INTEGER,
             FOREIGN KEY (order_id)
                 REFERENCES purchase_order (order_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION,
             FOREIGN KEY (product_id)
                 REFERENCES product (product_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION
             )'''

create_purchase_order_table = '''CREATE TABLE IF NOT EXISTS purchase_order (
             order_id INTEGER PRIMARY KEY,
             customer_id INTEGER,
             date TEXT NOT NULL,
             FOREIGN KEY (customer_id)
                 REFERENCES customer (customer_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION
             )'''


c.execute(create_product_table)
c.execute(create_customer_table)
c.execute(create_order_item_table)
c.execute(create_purchase_order_table)

In [ ]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .

import pandas as pd

#load the data from product csv into product table
product_df = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/product.csv")
customer_df = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv")
order_item_df = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv")
purchase_order_df = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv")


,order_id,product_id,quantity
0,0,0,1
1,0,1,1
2,1,2,2
3,1,3,1


In [ ]:
product_df

,product_id,name,price
0,0,bicycle,400
1,1,helmet,45
2,2,gloves,23
3,3,chain,48


In [ ]:
customer_df

,customer_id,name,email
0,0,Wendy Lee,wlee@bike.com
1,1,Jason Brown,jb@speed.com
2,2,Harry Potter,hp@hogwarts.edu
3,3,Godric Gryffindor,gg@hogwards.edu


In [ ]:
order_item_df

,order_id,product_id,quantity
0,0,0,1
1,0,1,1
2,1,2,2
3,1,3,1


In [ ]:
purchase_order_df

,order_id,customer_id,date
0,0,0,2020-10-19
1,1,0,2020-10-20
2,2,1,2020-10-20


In [ ]:
# insert the data from product_df dataframe to database table product
product_df.to_sql('product', conn, if_exists='append', index=False)
customer_df.to_sql('customer', conn, if_exists='append', index=False)
order_item_df.to_sql('order_item', conn, if_exists='append', index=False)
purchase_order_df.to_sql('purchase_order', conn, if_exists='append', index=False)


3

In [ ]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .

conn.commit() # commit the current transaction

In [ ]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .

c.execute("SELECT * FROM product")
product_results = c.fetchall()
print(product_results)

c.execute("SELECT * FROM customer")
customer_results = c.fetchall()
print(customer_results)

c.execute("SELECT * FROM order_item")
order_item_results = c.fetchall()
print(order_item_results)

c.execute("SELECT * FROM purchase_order")
purchase_order_results = c.fetchall()
print(purchase_order_results)

[(0, 'bicycle', 400.0), (1, 'helmet', 45.0), (2, 'gloves', 23.0), (3, 'chain', 48.0)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]


In [ ]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Query 1: ??? - Customer: Wendy Lee
# Query 2: ??? - Customer: Jason Brown
# Your code here . . .

#Query 1: Wendy Lee
query_1 = """SELECT purchase_order.date, product.name, order_item.quantity, customer.name
                FROM customer
                JOIN purchase_order ON purchase_order.customer_id = customer.customer_id
                JOIN order_item ON order_item.order_id = purchase_order.order_id
                JOIN product ON product.product_id = order_item.product_id
                WHERE customer.name='Wendy Lee'
                """
c.execute(query_1)
query_1_results = c.fetchall()
print("Query 1 Results:\n")
print(query_1_results)
#She has two orders (0 and 1), which seems to match with the items in those orders

#Query 2: Jason Brown
query_2 = """SELECT purchase_order.date, product.name, order_item.quantity, customer.name
                FROM customer
                JOIN purchase_order ON purchase_order.customer_id = customer.customer_id
                JOIN order_item ON order_item.order_id = purchase_order.order_id
                JOIN product ON product.product_id = order_item.product_id
                WHERE customer.name='Jason Brown'
                """
c.execute(query_2)
query_2_results = c.fetchall()
print("\nQuery 2 Results:\n")
print(query_2_results)
#Empty because even though he has an order_id (2), there is no corresponding order_id (2) in the order_item table

Query 1 Results:

[('2020-10-19', 'bicycle', 1, 'Wendy Lee'), ('2020-10-19', 'helmet', 1, 'Wendy Lee'), ('2020-10-20', 'gloves', 2, 'Wendy Lee'), ('2020-10-20', 'chain', 1, 'Wendy Lee')]

Query 2 Results:

[]


In [ ]:
# Additional tasks
# Appending a new row to one of the tables
# Deleting row(s) with "XXX"

#append a row with XXX
append_xxx = """INSERT INTO customer VALUES
                 (24,'XXX','abc@xyz.com')"""

c.execute(append_xxx)

c.execute("SELECT * FROM customer")
with_xxx = c.fetchall()
print(with_xxx)

[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu'), (24, 'XXX', 'abc@xyz.com')]


In [ ]:
#delete a row with XXX
delete_xxx = """DELETE FROM customer WHERE customer.name='XXX'"""

c.execute(delete_xxx)

c.execute("SELECT * FROM customer")
without_xxx = c.fetchall()
print(without_xxx)

[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
